In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import folium

/Users/benj/code/cdk1/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [11]:
# read the source dataset
data = pd.read_csv('../data/source/df_ch.csv', delimiter=';')

## CSV with yearly summed up precipation

In [12]:
# create a dataframe with the summed up yearly precipation
from datetime import datetime

yearly_df = data.copy()
yearly_df['DAY'] = yearly_df['DAY'].map(lambda x: datetime.strptime(str(x), '%Y%m%d'))
yearly_df['YEAR'] = yearly_df['DAY'].dt.year

yearly_df = yearly_df[
    (yearly_df['YEAR'] >= 1979) &
    (yearly_df['YEAR'] <= 2020)
    ]

out = yearly_df[['YEAR', 'LATITUDE', 'LONGITUDE', 'PRECIPITATION']].groupby(['YEAR', 'LATITUDE', 'LONGITUDE'],
                                                                            as_index=False).sum()
out.to_csv('../data/generated/precipation_yearly_sum.csv')


## CSV with monthly summed up precipation

In [13]:
# create a dataframe with the summed up monthly precipation
from datetime import datetime

df = data.copy()
df['DAY'] = df['DAY'].map(lambda x: datetime.strptime(str(x), '%Y%m%d'))
df['MONTH'] = df['DAY'].dt.strftime('%Y%m')
df['YEAR'] = df['DAY'].dt.year

df = df[
    (df['YEAR'] >= 1979) &
    (df['YEAR'] <= 2020)
    ]

out = df[['MONTH', 'LATITUDE', 'LONGITUDE', 'PRECIPITATION']].groupby(['MONTH', 'LATITUDE', 'LONGITUDE'],
                                                                      as_index=False).sum()
out.to_csv('../data/generated/precipation_monthly_sum.csv')

# Monthly Precipation in relation to global warming

In [16]:
from datetime import datetime

# load the dataframe with the global mean temp
global_mean_temp_df = pd.read_csv('../data/source/global_mean_temp.csv', delimiter=',')

df = data.copy()
df['DAY'] = df['DAY'].map(lambda x: datetime.strptime(str(x), '%Y%m%d'))
df['MONTH'] = df['DAY'].dt.strftime('%Y%m')
df['YEAR'] = df['DAY'].dt.year

df = df[['MONTH', 'YEAR', 'LATITUDE', 'LONGITUDE', 'PRECIPITATION']]


out = df.groupby(['MONTH', 'YEAR', 'LATITUDE', 'LONGITUDE'], as_index=False).sum()

result = pd.merge(out, global_mean_temp_df, on='YEAR', how='outer')

result = result[
    (result['YEAR'] >= 1979) &
    (result['YEAR'] <= 2020)
    ]

result.to_csv('../data/generated/precipation_monthly_sum_with_global_mean_temp.csv')

# Regularity of precipation by month 

In [6]:
df = data.copy()
df['DAY'] = df['DAY'].map(lambda x: datetime.strptime(str(x), '%Y%m%d'))
df['MONTH'] = df['DAY'].dt.strftime('%m')
df['YEAR'] = df['DAY'].dt.year

out = df[['MONTH', 'YEAR', 'LATITUDE', 'LONGITUDE', 'PRECIPITATION']].groupby(
    ['YEAR', 'MONTH', 'LATITUDE', 'LONGITUDE'], as_index=False).sum()
out = out[
    (out['YEAR'] >= 1979) &
    (out['YEAR'] <= 2020)
    ]
out.to_csv('../data/generated/precipation_monthly_regularity.csv')

# Floods grouped by month for switzerland

In [10]:
floods = pd.read_csv('../data/source/HANZE_events.csv', delimiter=',')

filtered_floods = floods[
    (floods['Year'] >= 1979) &
    (floods['Year'] <= 2020) &
    (floods['Cause'].to_string().__contains__('rainfall'))]

filtered_floods = filtered_floods.groupby(['Year']).size().reset_index(name='count')
filtered_floods[['Year', 'count']].to_csv('../data/generated/flooding_events_ch_filtered.csv')

data = pd.read_csv('../data/source/df_ch.csv', delimiter=';')
yearly_df = data.copy()
yearly_df['YEAR'] = yearly_df['DAY'].map(lambda x: datetime.strptime(str(x), '%Y%m%d').year)

out = yearly_df[['YEAR', 'PRECIPITATION']].groupby(['YEAR'], as_index=False).sum()

result = pd.merge(out, filtered_floods, left_on='YEAR', right_on='Year', how='outer')
result = result[['YEAR', 'PRECIPITATION', 'count']].fillna(0)
result.to_csv('../data/generated/flooding_events_ch_precipation_yearly.csv')
